## Dependencies

In [1]:
! pip install sentence_transformers
! pip install pyresparser
! pip install nltk
! pip install python-docx
#! pip install spacy 
!pip install spacy==2.3.5

!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

#! pip install exceptions
#! pip install resume-parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 4.8 MB/s 


## Imports

In [2]:
## Google Colab Specific
from google.colab import drive 
from google.colab import files

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Parsing a Resume

In [3]:

## For Parsing Resume
import nltk
nltk.download('stopwords')
from pyresparser import ResumeParser
import os
from docx import Document
 
filed='/content/drive/MyDrive/Colab Notebooks/Resume.pdf'

try:
    doc = Document()
    with open(filed, 'r') as file:
        doc.add_paragraph(file.read())
    doc.save("text.docx")
    data = ResumeParser('text.docx').get_extracted_data()
    print(data['skills'])
    res_inp=','.join(data['skills'])
except:
    data = ResumeParser(filed).get_extracted_data()
    print(data['skills'])
    res_inp=','.join(data['skills'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


['Android', 'Machine learning', 'Research', 'Matplotlib', 'C', 'Design', 'Parser', 'Js', 'Mobile', 'Java', 'Ai', 'Css', 'Communication', 'Javascript', 'Email', 'R', 'Django', 'Pandas', 'Windows', 'Html', 'Process', 'Selenium', 'Technical', 'Html5', 'Analysis', 'Linux', 'Flask', 'Computer science', 'Statistics', 'Operating systems', 'Php', 'Xml', 'Coaching', 'Tableau', 'Seaborn', 'Engineering', 'Certification', 'Github', 'Python', 'Programming', 'Sql', 'Database', 'Analyze']


## Parsing Job Descriptions for relevant skill sets

### Creating a job profile

In [4]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

def job_profile(nlp, matcher,text,application_subject):
	"""
	This funciton creates a profile of one sample row 
	in the dataset

	:param nlp: nlp load object with phrasematcher already initialized
	:param matcher: Custom PhraseMatcher object
	:param text: Resume/JD text data
	:param application_subject: company/applicant's name 
	:return: returns the particular profile in a dataframe format
	"""	

	# Get the matches from the text
	doc = nlp(text)
	matches = matcher(doc)
	
	# Create a dataframe to return
	d = []
	for match_id, start, end in matches:
		rule_id = nlp.vocab.strings[match_id]  # get the Skill, eg: 'Machine Learning'
		span = doc[start : end]  # get the Sub-skill, eg: 'Regression'
		d.append((rule_id, span.text))
	data = []
	for each,count in Counter(d).items():
		data.append([application_subject,*each,count])
	dataf = pd.DataFrame(data,columns=['Company/Candidate Name','Skill','Sub-skill','Count'])
	return(dataf)

### Creating a Course Profile

In [5]:
def course_profile(nlp, matcher,text,application_subject):
	"""
	This funciton creates a profile of one sample row 
	in the dataset

	:param nlp: nlp load object with phrasematcher already initialized
	:param matcher: Custom PhraseMatcher object
	:param text: Resume/JD text data
	:param application_subject: company/applicant's name 
	:return: returns the particular profile in a dataframe format
	"""	

	# Get the matches from the text
	doc = nlp(text)
	matches = matcher(doc)
	
	# Create a dataframe to return
	d = []
	for match_id, start, end in matches:
		rule_id = nlp.vocab.strings[match_id]  # get the Skill, eg: 'Machine Learning'
		span = doc[start : end]  # get the Sub-skill, eg: 'Regression'
		d.append((rule_id, span.text))
	data = []
	for each,count in Counter(d).items():
		data.append([application_subject,*each,count])
	dataf = pd.DataFrame(data,columns=['Course Name','Skill','Sub-skill','Count'])
	return(dataf)

### Creating a Parser


In [6]:
import pandas as pd
from spacy.matcher import PhraseMatcher

def init_parser_func(nlp, SKILL_FILE_PATH, file_type = 'csv'):
	"""
	This function initializes the nlp PhraseMatcher
	and reads the skill-set data file in an excel/csv format
	
	:param nlp: It is an NLP load instance, ex: nlp.load("en_core_web_sm")
	:param SKILL_FILE_PATH: Path for the data file
	:param file_type: csv/excel, default = csv
	:return: Returns a PhraseMatcher object.
	"""

	if file_type == 'csv':
		keyword_ = pd.read_csv(SKILL_FILE_PATH)
	else:
		try:
			keyword_ = pd.read_excel(SKILL_FILE_PATH)
			print("Success!")
		except Exception as e:
			print(e)

	matcher = PhraseMatcher(nlp.vocab)
	for each in keyword_.columns: 
		matcher.add(each, None, *[nlp(text) for text in keyword_[each].dropna(axis = 0)])

	return matcher

### Reading the skills and input job data and parsing relevant skillsets

In [8]:
import pandas as pd
import spacy
#from func import *
#from constants import *
#from init_parser import init_parser_func
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')


if __name__ == '__main__':

  print('hello world')
	
matcher = init_parser_func(nlp, '/content/drive/MyDrive/Colab Notebooks/Skillset.xlsx', file_type="excel")
 
final_database = pd.DataFrame()
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data.xlsx')
df = df.rename(columns={df.columns[0]: 'Company'})
df = df.rename(columns={df.columns[1]: 'Text'})
	
for each in tqdm(range(len(df))):

  text = df.loc[each,'Text']
  application_subject = df.loc[each,'Company']
  data = job_profile(nlp,matcher,text,application_subject)
  final_database = final_database.append(data)
      
final_agg_sub=pd.DataFrame(final_database.groupby('Company/Candidate Name')['Sub-skill'].apply(list))
final_agg_sub.reset_index(inplace=True)
final_agg_sup=pd.DataFrame(final_database.groupby('Company/Candidate Name')['Skill'].apply(list))
final_agg_sup.reset_index(inplace=True)
output_df=final_agg_sup.merge(final_agg_sub, how='left')
output_df['Skill']=output_df.Skill.apply( lambda x : ",".join(x))
output_df['Sub-skill']=output_df['Sub-skill'].apply( lambda x : ",".join(x))
output_df['input']=output_df['Skill']+','+output_df['Sub-skill']

hello world
Success!


100%|██████████| 14/14 [00:01<00:00, 12.86it/s]


## Recommending Jobs

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

### Getting Cosine Similarity 

In [10]:

X = np.array(output_df.input)
text_data = X
text_data=np.append(res_inp,text_data)
model = SentenceTransformer('distilbert-base-nli-mean-tokens')


embeddings = model.encode(text_data, show_progress_bar=True)

embed_data = embeddings
X = np.array(embed_data)
cos_sim_data = pd.DataFrame(cosine_similarity(X))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
cos_sim_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,0.944827,0.929510,0.831831,0.640770,0.824031,0.927603,0.921179,0.910557,0.830753,0.158461,0.908171,0.847196,0.912179,0.904821
1,0.944827,1.000000,0.962964,0.877982,0.642902,0.833506,0.937584,0.943281,0.925286,0.844667,0.155138,0.917670,0.846127,0.956527,0.932332
2,0.929510,0.962964,1.000000,0.901881,0.625247,0.822392,0.933159,0.897818,0.879603,0.841875,0.132585,0.897153,0.826951,0.961242,0.957213
3,0.831831,0.877982,0.901881,1.000000,0.631867,0.837539,0.868991,0.799457,0.802204,0.709470,0.173328,0.804398,0.786897,0.925044,0.891591
4,0.640770,0.642902,0.625247,0.631867,1.000000,0.752644,0.679881,0.714270,0.757152,0.649153,0.429792,0.687265,0.657816,0.685998,0.716900
5,0.824031,0.833506,0.822392,0.837539,0.752644,1.000000,0.894018,0.824462,0.885600,0.739180,0.246097,0.824816,0.775284,0.854724,0.852461
6,0.927603,0.937584,0.933159,0.868991,0.679881,0.894018,1.000000,0.869354,0.888692,0.786699,0.216656,0.907521,0.823343,0.918825,0.893585
7,0.921179,0.943281,0.897818,0.799457,0.714270,0.824462,0.869354,1.000000,0.968826,0.920575,0.220889,0.935103,0.870993,0.918228,0.926774
8,0.910557,0.925286,0.879603,0.802204,0.757152,0.885600,0.888692,0.968826,1.000000,0.874597,0.248840,0.921341,0.862398,0.912339,0.920539
9,0.830753,0.844667,0.841875,0.709470,0.649153,0.739180,0.786699,0.920575,0.874597,1.000000,0.201928,0.924413,0.816257,0.862662,0.893808


In [12]:
def subtractone(list):
  new_list=[]
  for elem in list:
    new_list.append(elem-1)

  return new_list

In [13]:
index_recomm =cos_sim_data.loc[0].sort_values(ascending=False).index.tolist()[1:6]
index_recomm=subtractone(index_recomm)
job_recomm =  output_df['Company/Candidate Name'].loc[index_recomm].values
result = {'Jobs':job_recomm,'Index':index_recomm}

output_df.loc[index_recomm]

,Company/Candidate Name,Skill,Sub-skill,input
0,Company_1,"Maths,Coding,ML/AI,ML/AI,DevOps,Coding,WebD,Co...","Algorithms,Algorithms,Regression,Random Forest...","Maths,Coding,ML/AI,ML/AI,DevOps,Coding,WebD,Co..."
1,Company_10,"Business,DevOps,Business,DevOps,Business,DevOp...","performance metrics,Google Analytics,Google An...","Business,DevOps,Business,DevOps,Business,DevOp..."
5,Company_14,"Soft_Skills,WebD,DevOps,Business,Coding,WebD,C...","SEO,client,Google Analytics,Google Analytics,H...","Soft_Skills,WebD,DevOps,Business,Coding,WebD,C..."
6,Company_2,"ML/AI,ML/AI,Maths,ML/AI,Coding,WebD,WebD,DevOp...","NLP,Computer Vision,Statistics,Keras,Python,Py...","ML/AI,ML/AI,Maths,ML/AI,Coding,WebD,WebD,DevOp..."
12,Company_8,"DevOps,Coding,WebD,DevOps,Coding,Coding,Busine...","Hive,Python,Python,Spark,R,Scala,Marketing,SQL...","DevOps,Coding,WebD,DevOps,Coding,Coding,Busine..."


## Lets Recommend Courses!

In [14]:
if __name__ == '__main__':
  print("main")


final_database = pd.DataFrame()
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data.xlsx')
df = df.rename(columns={df.columns[0]: 'Course Title'})
df = df.rename(columns={df.columns[1]: 'Description'})
	
for each in tqdm(range(len(df))):

  desc = df.loc[each,'Description']
  application_subject = df.loc[each,'Course Title']
  data = course_profile(nlp,matcher,desc,application_subject)
  final_database = final_database.append(data)
      
final_agg_sub=pd.DataFrame(final_database.groupby('Course Name')['Sub-skill'].apply(list))
final_agg_sub.reset_index(inplace=True)
final_agg_sup=pd.DataFrame(final_database.groupby('Course Name')['Skill'].apply(list))
final_agg_sup.reset_index(inplace=True)
output_df=final_agg_sup.merge(final_agg_sub, how='left')
output_df['Skill']=output_df.Skill.apply( lambda x : ",".join(x))
output_df['Sub-skill']=output_df['Sub-skill'].apply( lambda x : ",".join(x))
output_df['input']=output_df['Skill']+','+output_df['Sub-skill']

main


100%|██████████| 1999/1999 [00:44<00:00, 45.13it/s]


In [15]:
index_recomm = cos_sim_data.loc[0].sort_values(ascending=True).index.tolist()[1:6]
index_recomm = subtractone(index_recomm)
course_recomm =  output_df['Course Name'].values
result = {'Courses':course_recomm,'Index':index_recomm}
output_df.loc[index_recomm]

,Course Name,Skill,Sub-skill,input
3,Advanced Modeling for Discrete Optimization,Analytics,Optimization,"Analytics,Optimization"
4,Advanced Presentation Skills: You Can Speak Wi...,Soft_Skills,Presentation,"Soft_Skills,Presentation"
8,"Algebra II - DVD, digital video course",Maths,Algebra,"Maths,Algebra"
2,ActiveCampaign QuickStart - Marketing Automation,Business,Marketing,"Business,Marketing"
11,Backend in Node.js,"Coding,DevOps","JavaScript,Node.js","Coding,DevOps,JavaScript,Node.js"
